# Pipelines

### Not using pipelines

In [ ]:
# Pipelines chains together multiple steps so that the output of each step is used as input to the next step.
# Pipelines makes it easy to apply the same preprocessing to train and test

import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer     # handles missing data
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer

In [ ]:
df = pd.read_csv('train.csv')
df.head()

In [ ]:
# dropping unrequired columns
df = df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])
print(df.isna().sum())
df['Age'] = df['Age'].fillna(df['Age'].mean())   # as age contains 177 NaN values, hence storing the  mean in those empty cells
df = df.dropna(subset=['Embarked'])              # removing the NaN values from Embarked 
print()


print(df.isna().sum())

x_train, x_test, y_train, y_test = train_test_split(df.drop(columns='Survived'), df['Survived'], test_size=0.2, random_state=0)

# imputation
# i have removed all the 0 values from Age and Embarked by using mean in the NaN cells, hence there is no need to use imputaion
# still we will be using it for better understanding of the code
si_age = SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.transform(x_test[['Age']])
x_test_embarked = si_embarked.transform(x_test[['Embarked']])


# one hot encoding
ohe_sex = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train[['Embarked']])

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test[['Embarked']])


# dropping the unrequired data
x_train_remains = x_train.drop(columns=['Sex', 'Age', 'Embarked'])
x_test_remains = x_test.drop(columns=['Sex', 'Age', 'Embarked'])

# concanating the data
x_train_transformed = np.concatenate((x_train_remains, x_train_age, x_train_sex, x_train_embarked), axis=1)
x_test_transformed = np.concatenate((x_test_remains, x_test_age, x_test_sex, x_test_embarked), axis=1)


# decision tree
clf = DecisionTreeClassifier()
clf.fit(x_train_transformed, y_train)

# prediction
y_pred = clf.predict(x_test_transformed)
print()
print(f"ACCURACY SCORE : {np.round(accuracy_score(y_test, y_pred), 2)*100} %")






In [ ]:
# converitng into dataframe

col = (
    list(x_train_remains.columns) +
    ['Age'] +
    list(ohe_sex.get_feature_names_out(['Sex'])) +
    list(ohe_embarked.get_feature_names_out(['Embarked']))
)

x_train_transformed = pd.DataFrame(x_train_transformed, columns=col)
x_train_transformed


Exporting the above data to the website

In [ ]:
import pickle

In [ ]:
pickle.dump(ohe_sex, open('models/ohe_sex.pkl', 'wb'))
pickle.dump(ohe_embarked, open('models/ohe_embarked.pkl', 'wb'))
pickle.dump(clf, open('models/clf.pkl', 'wb'))
pickle.dump(col, open('models/col.pkl', 'wb'))

# all the above files will be stored in the models folder which is inside the current folder
# go to the file "9_1 using pickle.ipynb", there you will learn how to use pickle 